# GitHub Setup & Data Import

In [46]:
!git clone https://github.com/vpaulino26/fish-predict

Cloning into 'fish-predict'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 33 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 23.75 MiB | 13.62 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [47]:
%cd fish-predict
%cd datasets
!unzip 22560_NEFSCFallFisheriesIndependentBottomTrawlData.zip

/content/fish-predict/datasets/fish-predict/datasets/fish-predict
/content/fish-predict/datasets/fish-predict/datasets/fish-predict/datasets
Archive:  22560_NEFSCFallFisheriesIndependentBottomTrawlData.zip
  inflating: 22560_SVDBS_CRUISES.csv  
  inflating: 22560_UNION_FSCS_SVBIO.csv  
  inflating: 22560_UNION_FSCS_SVCAT.csv  
  inflating: 22560_UNION_FSCS_SVLEN.csv  
  inflating: 22560_UNION_FSCS_SVSTA.csv  


In [48]:
# Imports:
import pandas as pd
from numpy import shape

# Part 1: Data Set Up & Cleaning

This section focuses on preparing the raw data for analysis and model training. The key steps involve:

1.  **Loading Data:** Importing the `22560_UNION_FSCS_SVCAT.csv` and `22560_UNION_FSCS_SVSTA.csv` files into pandas DataFrames.
2.  **Filtering Data:** Reducing the `svc_cat_df` to include only the top 10 most commercially caught species in Rhode Island, as listed below.
3.  **Dropping Irrelevant Columns:** Removing columns from both DataFrames that are not necessary for the analysis or model building.
4.  **Missing Data Adjustments:** Remove columns where more than half of the entries have a null value since they won't make good predictors.
5.  **Merging DataFrames:** Combining the filtered species data with the station data based on common identifiers to create a unified dataset for subsequent steps.

These are the top 10 commercial species in RI we want to focus on:
1. LONGFIN SQUID
2. SEA SCALLOP
3. AMERICAN LOBSTER
4. SHORTFIN SQUID (ILLEX)
5. SUMMER FLOUNDER
6. QUAHOG
7. SCUP
8. JONAH CRAB
9. BUTTERFISH
10. SILVER HAKE

Note: The species "SHORTFIN SQUID (ILLEX)" is referred to as "NORTHERN SHORTFIN SQUID" in the dataset.

### 1.1 Loading Data

In [77]:
# Import the two data sets and save them as DFs
svcat_df = pd.read_csv('22560_UNION_FSCS_SVCAT.csv', dtype={'CRUISE STRATUM': str, 'STATUS_CODE': str, 'CATCH_COMMENT': str}, low_memory=False)
svsta_df = pd.read_csv('22560_UNION_FSCS_SVSTA.csv', encoding='latin-1', low_memory=False)

print(f"SVCAT:{shape(svcat_df)} \nSVSTA:{shape(svsta_df)}\n")

display(svcat_df.head())
print("\n")
display(svsta_df.head())

SVCAT:(338180, 13) 
SVSTA:(21823, 94)



,CRUISE6,CRUISE,STRATUM,TOW,STATION,STATUS_CODE,ID,SVSPP,CATCHSEX,EXPCATCHNUM,EXPCATCHWT,SCIENTIFIC_NAME,CATCH_COMMENT
0,196307,6307,01260,1,1,10,196307012600010001,15,0,10.0,19.5,Squalus acanthias (spiny dogfish),
1,196307,6307,01260,1,1,10,196307012600010001,23,0,53.0,142.0,Leucoraja ocellata (winter skate),
2,196307,6307,01260,1,1,10,196307012600010001,26,0,11.0,7.7,Leucoraja erinacea (little skate),
3,196307,6307,01260,1,1,10,196307012600010001,31,0,1.0,0.0,Etrumeus teres (round herring),
4,196307,6307,01260,1,1,10,196307012600010001,33,0,2.0,0.5,Alosa pseudoharengus (alewife),


,CRUISE6,CRUISE,STRATUM,TOW,STATION,STATUS_CODE,ID,STATYPE,HAUL,GEARCOND,...,SURFTEMP,SURFSALIN,BOTTEMP,BOTSALIN,FULD,NO_DETAIL_SVSPP,BOTSPEED,WATCH_CHIEF_COMMENTS,STATION_COMMENTS,HABITAT_COMMENTS
0,196307,6307,01260,1,1,10,196307012600010001,1.0,2.0,3.0,...,11.0,NaN,10.1,NaN,NaN,NaN,0.0,,,
1,196307,6307,01260,2,2,10,196307012600020002,1.0,3.0,5.0,...,9.9,NaN,9.9,NaN,NaN,NaN,0.0,,,
2,196307,6307,01260,3,3,10,196307012600030003,1.0,2.0,3.0,...,9.9,NaN,8.1,NaN,NaN,NaN,0.0,,,
3,196307,6307,01260,4,4,10,196307012600040004,1.0,3.0,5.0,...,10.2,NaN,10.1,NaN,NaN,NaN,0.0,,,
4,196307,6307,01260,5,5,10,196307012600050005,1.0,2.0,3.0,...,10.0,NaN,9.4,NaN,NaN,NaN,0.0,,,


### 1.2 Filtering Data

In [78]:
# Filter out non top species
top_species = ['LONGFIN SQUID', 'SEA SCALLOP', 'AMERICAN LOBSTER', 'SHORTFIN SQUID', 'SUMMER FLOUNDER', 'QUAHOG', 'SCUP', 'JONAH CRAB', 'BUTTERFISH', 'SILVER HAKE']

# Filter only rows where the Species column contains any of these names
filtered_svcat = svcat_df[svcat_df["SCIENTIFIC_NAME"].apply(
    lambda x: any(SCIENTIFIC_NAME.lower() in str(x).lower() for SCIENTIFIC_NAME in top_species)
)]

# Compare the unfiltered and filtered dataframes to see that filtering was succesful
print(f"Unfiltered SVCAT:{shape(svcat_df)} \nFiltered SVCAT:{shape(filtered_svcat)}")

Unfiltered SVCAT:(338180, 13) 
Filtered SVCAT:(77275, 13)


### 1.3 Column Adjustments
- We will likely drop more columns from SVSTA as we discover which columns have the most entropy and aren't useful in the model in order to save on training time and complexity.

In [79]:
# Drop columns we won't use from SVCAT
filtered_svcat = filtered_svcat.drop(columns=['CATCHSEX', 'CATCH_COMMENT'])

# Drop columns we won't use from SVSTA
columns_to_drop_sv_sta = [
    'TYPE_CODE', 'TYPE_COMMENT', 'OPERATION_CODE', 'OPERATION_COMMENT',
    'GEAR_CODE', 'GEAR_COMMENT', 'ACQUISITION_CODE', 'ACQUISITION_COMMENT',
    'OTHGEAR', 'WATCH_CHIEF_COMMENTS', 'STATION_COMMENTS', 'HABITAT_COMMENTS'
]
filtered_svsta = svsta_df.drop(columns=columns_to_drop_sv_sta)

# Compare the unfiltered and filtered dataframes to see that filtering was succesful
print(f"Updated SVCAT:{shape(filtered_svcat)} \nUpdated SVSTA:{shape(filtered_svsta)}")

Updated SVCAT:(77275, 11) 
Updated SVSTA:(21823, 82)


### 1.4 Adjusting Missing Values
- Remove the columns where more than 50% of the entires don't have a value
  - 50% is a threshold, we can adjust the amount if we need to
  - SVCAT has no columns that meet this criteria

In [80]:
# Check for missing values in filtered_svcat
print("Missing values in filtered_svcat (over 50% missing):")
missing_svcat = filtered_svcat.isnull().sum() / len(filtered_svcat) * 100
if missing_svcat[missing_svcat > 50].empty:
  print("No columns with over 50% missing values in filtered_svcat")
else:
  display(missing_svcat[missing_svcat > 50])

print("\nMissing values in filtered_svsta (over 50% missing):")
# Check for missing values in filtered_svsta
missing_svsta = filtered_svsta.isnull().sum() / len(filtered_svsta) * 100
display(missing_svsta[missing_svsta > 50])

# Drop the columns that are in missing_svsta
cols_to_drop = missing_svsta[missing_svsta > 50].index.tolist()
filtered_svsta = filtered_svsta.drop(columns=cols_to_drop)
print(f"\nUpdated SVSTA shape after dropping columns with over 50% missing values: {shape(filtered_svsta)}")

Missing values in filtered_svcat (over 50% missing):
No columns with over 50% missing values in filtered_svcat

Missing values in filtered_svsta (over 50% missing):


,0
TOGA,73.463777
BEGEKVLOG,96.879439
ENDEKVLOG,96.879439
ENDLAT,54.140127
ENDLON,54.140127
DECDEG_ENDLAT,54.140127
DECDEG_ENDLON,54.140127
PITCH,53.365715
RPM,52.009348
DOPDISTW,76.891353



Updated SVSTA shape after dropping columns with over 50% missing values: (21823, 62)


### 1.5 Merging DataFrames
- Merge the DataFrames together on ID so that all the information is in one DF
- The columns that appear in both datasets always have identical data, so we can only keep the columns from one and avoid dealing with duplicates

In [105]:
# Merge the dataframes on 'ID'
merged_df = pd.merge(filtered_svcat, filtered_svsta, on='ID', suffixes=('_cat', '_sta'))

# Drop duplicate columns, keeping the ones from filtered_svcat (those without '_sta' suffix)
columns_to_drop = [col for col in merged_df.columns if col.endswith('_sta')]
merged_df = merged_df.drop(columns=columns_to_drop)

# Rename the _cat columns to remove suffix
merged_df.columns = merged_df.columns.str.replace('_cat', '')

# Display the merged dataframe
print(f"Merged dataframe shape: {shape(merged_df)}")
display(merged_df.head())

Merged dataframe shape: (77275, 66)


,CRUISE6,CRUISE,STRATUM,TOW,STATION,STATUS_CODE,ID,SVSPP,EXPCATCHNUM,EXPCATCHWT,...,BAROPRESS,WINDDIR,WINDSP,WAVEHGT,SWELLDIR,SWELLHGT,XBT,SURFTEMP,BOTTEMP,BOTSPEED
0,196307,6307,01260,1,1,10,196307012600010001,72,52.0,9.5,...,1012.0,250.0,6.0,0.0,NaN,NaN,3,11.0,10.1,0.0
1,196307,6307,01260,2,2,10,196307012600020002,72,35.0,18.1,...,1012.0,230.0,8.0,0.0,NaN,NaN,3,9.9,9.9,0.0
2,196307,6307,01260,3,3,10,196307012600030003,72,164.0,10.4,...,1012.0,200.0,6.0,0.0,NaN,NaN,3,9.9,8.1,0.0
3,196307,6307,01260,3,3,10,196307012600030003,503,7.0,0.0,...,1012.0,200.0,6.0,0.0,NaN,NaN,3,9.9,8.1,0.0
4,196307,6307,01260,4,4,10,196307012600040004,72,7.0,3.2,...,1012.0,220.0,6.0,0.0,NaN,NaN,3,10.2,10.1,0.0
